<font size="8"> **Adding environmental data from ACCESS-OM2-01 to unique crabeater observations** </font>  
In this notebook, we will extract environmental data from the ACCESS-OM2-01 model outputs and add it to our data frame containing unique crabeater sightings per month and grid cell (see `04_Creating_bio_data_masks.ipynb` for more information.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41793,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:46227,Total threads: 2
Dashboard: /proxy/35791/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:46257,


# Loading unique crabeater seal observations data frame

In [100]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Biological_Data/unique_crabeater_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,month,season_year,life_stage,decade,presence
0,2019-03-09T00:00:00Z,2019-03,-64.9028,-62.8643,Belgica_121,2019,East Pacific,Antarctic,-64.888,-62.85,3,autumn,in-between,2010,1
1,2019-03-13T00:00:00Z,2019-03,-65.1413,-64.1443,Belgica_121,2019,East Pacific,Antarctic,-65.142,-64.15,3,autumn,in-between,2010,1
2,2019-03-15T00:00:00Z,2019-03,-65.1402,-64.0595,Belgica_121,2019,East Pacific,Antarctic,-65.142,-64.05,3,autumn,in-between,2010,1
3,2019-03-15T00:00:00Z,2019-03,-65.0977,-63.9802,Belgica_121,2019,East Pacific,Antarctic,-65.100,-63.95,3,autumn,in-between,2010,1
4,2019-03-14T00:00:00Z,2019-03,-65.3383,-64.1300,Belgica_121,2019,East Pacific,Antarctic,-65.353,-64.15,3,autumn,in-between,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,2006-02-20T13:00:00Z,2006-02,-67.3064,73.1346,SCAR_seabirds,2006,Central Indian,Antarctic,-67.296,73.15,2,summer,moulting,2000,1
3236,2006-02-19T13:00:00Z,2006-02,-67.0479,69.9957,SCAR_seabirds,2006,Central Indian,Antarctic,-67.043,69.95,2,summer,moulting,2000,1
3237,2006-02-19T13:00:00Z,2006-02,-67.0665,69.9945,SCAR_seabirds,2006,Central Indian,Antarctic,-67.085,69.95,2,summer,moulting,2000,1
3238,2006-02-19T13:00:00Z,2006-02,-67.1395,69.9984,SCAR_seabirds,2006,Central Indian,Antarctic,-67.127,69.95,2,summer,moulting,2000,1


# Adding values for static variables only
Static variables referred to any physical variables that do not change over time (at least not during the time period of our interest). Examples include depth of the water column and distance to coastline. Given that we only have one value for these variables, the process of extracting data is relatively simple. We do not need to take into account the date observations were collected.

## Defining dictionary with information about static variables
This dictionary contains the column labels for each and the name of the files for each static variable to be included in our analysis. We will also define a variable containing the full path to the folder where all static variables are stored.

In [6]:
#Full path to static variables
base_dir_static = '/g/data/v45/la6889/Chapter2_Crabeaters/Static_Variables/'

#List of static variables
varDict = {'bottom_slope_deg': 'bathy_slope_GEBCO_2D.nc',
           'dist_shelf_km': 'distance_shelf.nc',
           'dist_coast_km': 'distance_coastline.nc',
           'depth_m': 'bathy_GEBCO_2D.nc'}

## Extracting data for each observation and adding it to a new column in crabeater data

In [101]:
#Getting coordinates from crabeater dataset
lat = xr.DataArray(crabeaters.latitude.values)
lon = xr.DataArray(crabeaters.longitude.values)

#Looping through dictionary keys
for var in varDict:
    #Creating full path to file of interest
    file_path = os.path.join(base_dir_static, varDict[var])
    #Load as raster
    ras = xr.open_dataarray(file_path)
    rename_var = ras.name
    #Extracting values
    ras_sub = ras.sel(xt_ocean = lon, yt_ocean = lat, method = 'nearest')
    #Turning into data frame and rounding all columns to 3 decimal places
    ras_df = ras_sub.to_dataframe().round(3).rename(columns = {rename_var: var})
    #Adding to crabeater observations data frame
    crabeaters = pd.merge(crabeaters, ras_df, on = ['yt_ocean', 'xt_ocean'], how = 'left', sort = True).drop_duplicates()
    
#Checking results
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,month,season_year,life_stage,decade,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,2,summer,moulting,2000,1,89.980,606.014,32.155,176.033005
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,1,summer,moulting,2010,1,89.730,383.203,41.999,1064.319946
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,1,summer,moulting,2000,1,89.943,113.770,4.724,612.181030
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,1,summer,moulting,2000,1,89.898,113.279,2.642,656.099976
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,2,summer,moulting,2020,1,89.871,603.311,21.027,519.267029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897,1989-11-05T10:00:00Z,1989-11,-60.066700,70.200000,SCAR_seabirds,1989,Central Indian,Subantarctic,-60.047,70.25,11,autumn,weaning,1980,1,89.978,478.562,622.374,4235.085938
3898,2008-12-26T00:00:00Z,2008-12,-59.640000,9.115000,GBIF_IRSNB_Belgian Antarctic Research Expediti...,2008,Atlantic,Antarctic,-59.645,9.15,12,summer,weaning,2000,1,89.958,844.113,926.685,5462.943848
3899,1989-11-05T02:00:00Z,1989-11,-59.483300,71.866700,SCAR_seabirds,1989,Central Indian,Subantarctic,-59.493,71.85,11,autumn,weaning,1980,1,89.865,531.076,692.923,4498.345215
3900,2013-11-12T00:00:00Z,2013-11,-57.826121,-26.346061,GBIF_NA_Observations,2013,Atlantic,Antarctic,-57.829,-26.35,11,autumn,weaning,2010,1,NaN,1443.071,1067.077,506.523987


## Saving data frame with static variables
Given that the dynamic variables take some time to extract. We will save intermediary results to avoid having to extract them again.

In [102]:
crabeaters.to_csv('../Environmental_Data/unique_crabeater_human-obs_static_env.csv', index = False)

# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [16]:
crabeaters = pd.read_csv('../Environmental_Data/unique_crabeater_human-obs_static_env.csv')
#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,month,season_year,life_stage,decade,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,2,summer,moulting,2000,1,89.980,606.014,32.155,176.033
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,1,summer,moulting,2010,1,89.730,383.203,41.999,1064.320
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,1,summer,moulting,2000,1,89.943,113.770,4.724,612.181
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,1,summer,moulting,2000,1,89.898,113.279,2.642,656.100
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,2,summer,moulting,2020,1,89.871,603.311,21.027,519.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,1989-11-05T10:00:00Z,1989-11,-60.066700,70.200000,SCAR_seabirds,1989,Central Indian,Subantarctic,-60.047,70.25,11,autumn,weaning,1980,1,89.978,478.562,622.374,4235.086
3236,2008-12-26T00:00:00Z,2008-12,-59.640000,9.115000,GBIF_IRSNB_Belgian Antarctic Research Expediti...,2008,Atlantic,Antarctic,-59.645,9.15,12,summer,weaning,2000,1,89.958,844.113,926.685,5462.944
3237,1989-11-05T02:00:00Z,1989-11,-59.483300,71.866700,SCAR_seabirds,1989,Central Indian,Subantarctic,-59.493,71.85,11,autumn,weaning,1980,1,89.865,531.076,692.923,4498.345
3238,2013-11-12T00:00:00Z,2013-11,-57.826121,-26.346061,GBIF_NA_Observations,2013,Atlantic,Antarctic,-57.829,-26.35,11,autumn,weaning,2010,1,NaN,1443.071,1067.077,506.524


## Accessing ACCESS-OM2-01 model outputs
We will create a new `cosima cookbook` session to load the model outputs of interest, and we will also create a dictionary that contains useful information related to data extraction.

In [4]:
#Creating new COSIMA cookbook session
session = cc.database.create_session()

#Creating dictionary with useful information
varDict = {'model': 'ACCESS-OM2-01',
           #ACCESS-OM2-01 cycle 4 (1958-2018)
           'exp': '01deg_jra55v140_iaf_cycle4',
           #ACCESS-OM2-01 cycle 4 extension (2018-2022)
           'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
           #Temporal resolution
           'freq': '1 monthly',
           #Output folder
           'base_out': '../Environmental_Data'}

## Loading data frame with ACCESS-OM2-01 outputs
We can use this data frame to find the variable names for the environmental factors that we know are influential for the distribution of crabeater seals.

In [102]:
#Loading data frame with model outputs
# var_acc = cc.querying.get_variables(session, experiment = varDict['exp_ext'], frequency = '1 monthly')

#Searching data frame for variables of interest
var_acc[var_acc.name.str.contains('salt')]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
58,fsalt_ai_m,salt flux ice to ocean,kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
59,fsalt_m,salt flux ice to ocn (cpl),kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
96,salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-3d-salt-1-monthly-mean-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
97,salt_xflux_adv,rho*dzt*dyt*u*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_xflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
98,salt_yflux_adv,rho*dzt*dxt*v*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_yflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
105,sfc_salt_flux_coupler,sfc_salt_flux_coupler: flux from the coupler,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_couple...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
106,sfc_salt_flux_ice,sfc_salt_flux_ice,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_ice-1-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
107,sfc_salt_flux_restore,sfc_salt_flux_restore: flux from restoring term,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_restor...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
120,surface_salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-2d-surface_salt-1-month...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00


## Completing dictionary with useful variables
Now that we identified the correct name for the variable of our interest, we can complete our dictionary.

In [5]:
#Variable name in the model
varDict['var_mod'] = 'surface_salt'
#Name of column where we will store the extracted data
varDict['var_short_name'] = 'SSS_psu'
#Defining if this variable is related to sea ice or not
varDict['ice_data'] = False
#Checking final dictionary
varDict

{'model': 'ACCESS-OM2-01',
 'exp': '01deg_jra55v140_iaf_cycle4',
 'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
 'freq': '1 monthly',
 'base_out': '../Environmental_Data',
 'var_mod': 'surface_salt',
 'var_short_name': 'SSS_psu',
 'ice_data': False}

## Loading data from ACCESS-OM2-01
ACCESS-OM2-01 has four different cycles available with a temporal range of 60 years (1958-2018), but only cycle 4 includes biogeochemical (BGC) outputs. Since we are interested in examining the effect of some BGC variables on crabeater seals, we will use the fourth cycle in this project. Another special feature of cycle 4 is that it was extended until December 2022. Outputs from cycle 4 are available through two experiments: `01deg_jra55v140_iaf_cycle4` which goes from 1958 to 2018, and `01deg_jra55v140_iaf_cycle4_jra55v150_extension` that includes outputs from 2019 to 2022. The crabeater dataset has temporal range between 1978 and 2022, and this is the reason why we are using the two experiment of cycle 4.  
  
In the chunk below, we load the ACCESS-OM2-01 data, correct longitudes so they range between -180 and +180, and apply a transformation to temperature outputs (`temp`) only because the original units are in Kelvin and we need them in $^{\circ}C$.

In [8]:
#Loading data from fourth cycle (temporal range 1958 to 2018)
var_df = uf.getACCESSdata_SO(varDict['var_mod'], '1978-01', '2019-01', 
                              freq = varDict['freq'], ses = session, minlat = -80,
                              exp = varDict['exp'], ice_data = varDict['ice_data'])
#Loading data from fourth cycle extension (2019 to 2022)
var_df_ext = uf.getACCESSdata_SO(varDict['var_mod'], '2019-01', '2023-01', 
                              freq = varDict['freq'], ses = session, minlat = -80,
                              exp = varDict['exp_ext'], ice_data = varDict['ice_data'])

#Concatenating both data arrays into one
var_df = xr.concat([var_df, var_df_ext], dim = 'time')

#Transforming longitudes so their range is +/-180 degrees
var_df = uf.corrlong(var_df)

#If temperature data, transform from Kelvin to degrees C
if var_df.name == 'temp':
    var_df = var_df-273.15

#Removing duplicate variable
del var_df_ext

#Show results
var_df.isel(time = 0)

<xarray.DataArray 'surface_salt' (yt_ocean: 713, xt_ocean: 3600)>
dask.array<getitem, shape=(713, 3600), dtype=float32, chunksize=(513, 720), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
    time      datetime64[ns] 1977-12-16T12:00:00
Attributes: (12/13)
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    ...             ...
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2022-04-27
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

### *Optional: Subsetting surface layer data*
For some ocean variables, we need to subset data to extract surface values or bottom values. Subsetting data for the surface layer is an easy process, we simply need to select the first depth bin available. The `st_ocean` dimension contains the depth bins.

In [8]:
#Selecting the first depth available in the model (i.e. surface layer)
var_df_surf = var_df.isel(st_ocean = 0)

#Removing depth dimension
var_df_surf = var_df_surf.squeeze().drop('st_ocean')

#Checking results - dataset has three dimensions instead of the original four
var_df_surf

<xarray.DataArray 'temp' (time: 541, yt_ocean: 713, xt_ocean: 3600)>
dask.array<getitem, shape=(541, 713, 3600), dtype=float32, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00

### *Optional: Subsetting bottom data*
Subsetting data for the bottom layer is not as straightforward as for the surface. This is because the bathymetry is not the same across the Southern Ocean. To identify the correct depth bin that contains the deepest values for a particular grid cell.  
  
We have included a function called `extract_bottom_layer` in the `UsefulFunctions.py` script that extracts data for the bottom layer of any variables with a depth dimension (`st_ocean`). For more details refer to the script.  

In [22]:
var_df = uf.extract_bottom_layer(var_df)
#Checking results
var_df

## Loading dynamic variables derived from calculations
These variables are not directly available in ACCESS-OM2-01, but they have been calculated from model outputs. Refer to folder `02_Environmental_Data` to see the full details of each calculation.

In [71]:
var_df = xr.open_mfdataset(sorted(glob('/g/data/v45/la6889/Chapter2_Crabeaters/SeaIce/LongTerm_PackIce/*.nc'))).aice_m
var_df

<xarray.DataArray 'aice_m' (time: 540, yt_ocean: 713, xt_ocean: 3600)>
dask.array<concatenate, shape=(540, 713, 3600), dtype=float64, chunksize=(12, 713, 3600), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * time      (time) datetime64[ns] 1978-01-01 1978-02-01 ... 2022-12-01

## Extracting environmental data
We will use the `latitude` and `longitude` columns together with the `event_date` column from the crabeater seal observations to find the corresponding grid cell in the model outputs and extract the value of the environmental factor of our interest.

In [ ]:
#Getting coordinates from the crabeater data
lat = xr.DataArray(crabeaters.latitude)
lon = xr.DataArray(crabeaters.longitude)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{x.date}-16'), axis = 1))

In [18]:
#Extracting data
# var_sub = var_df_surf.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')
var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')

#Transforming to data frame
var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
#Adding year and month
var_pd['year'] = var_pd.time.dt.year
var_pd['month'] = var_pd.time.dt.month
#Removing time column that is no longer needed
var_pd.drop(columns = 'time', inplace = True)
#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Renaming variable to be added to crabeater data
var_pd.rename(columns = {varDict['var_mod']: varDict['var_short_name']}, inplace = True)
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_short_name']).columns.tolist()

#Checking results
cols; var_pd

,xt_ocean,yt_ocean,bottom_temp_deg,year,month
dim_0,,,,,
1467,108.95,-65.058,-0.444885,1978,1
1514,108.95,-65.015,-0.443085,1978,1
1189,109.65,-65.353,-0.760895,1978,1
1255,109.65,-65.269,-0.495453,1978,1
1704,109.65,-64.846,-0.445282,1978,1
...,...,...,...,...,...
1729,-62.65,-64.803,0.000000,2022,11
645,-66.85,-66.916,-0.524384,2022,12
646,-66.75,-66.916,-0.524719,2022,12


In [20]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,...,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,freez_pot_Wm2,SIT_m,SIC,SST_deg,bottom_temp_deg
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,...,1,89.980,606.014,32.155,176.033,-11441.6450,0.000992,0.012192,-0.398438,-1.389679
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,...,1,89.730,383.203,41.999,1064.320,-37128.7270,0.000500,0.000326,2.801361,-1.864105
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,...,1,89.943,113.770,4.724,612.181,-11849.3600,0.276383,0.107635,-0.370117,1.146729
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,...,1,89.898,113.279,2.642,656.100,-11528.4590,0.305247,0.123588,-0.404816,1.145599
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,...,1,89.871,603.311,21.027,519.267,-26964.8730,0.000602,0.000387,1.423065,-1.873322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16780449,1989-11-05T10:00:00Z,1989-11,-60.066700,70.200000,SCAR_seabirds,1989,Central Indian,Subantarctic,-60.047,70.25,...,1,89.978,478.562,622.374,4235.086,-5222.8315,0.186384,0.188992,-1.193298,-0.517700
16780450,2008-12-26T00:00:00Z,2008-12,-59.640000,9.115000,GBIF_IRSNB_Belgian Antarctic Research Expediti...,2008,Atlantic,Antarctic,-59.645,9.15,...,1,89.958,844.113,926.685,5462.944,-3173.7544,0.559869,0.547681,-1.436432,-0.770233
16780451,1989-11-05T02:00:00Z,1989-11,-59.483300,71.866700,SCAR_seabirds,1989,Central Indian,Subantarctic,-59.493,71.85,...,1,89.865,531.076,692.923,4498.345,-4823.6560,0.164699,0.178045,-1.240387,-0.523804
16780452,2013-11-12T00:00:00Z,2013-11,-57.826121,-26.346061,GBIF_NA_Observations,2013,Atlantic,Antarctic,-57.829,-26.35,...,1,NaN,1443.071,1067.077,506.524,-6468.0820,0.007129,0.005647,-1.099792,0.045715


In [21]:
#Create file path where data will be saved
file_out = os.path.join(varDict['base_out'], 'unique_crabeater_obs_all_env.csv')
#Saving as csv file
crabeaters.to_csv(file_out, index = False)

In [7]:
file_out = os.path.join(varDict['base_out'], 'unique_crabeater_obs_all_env.csv')
crabeaters = pd.read_csv(file_out)
crabeaters.head()

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,...,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,freez_pot_Wm2,SIT_m,SIC,SST_deg,bottom_temp_deg
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,...,1,89.980,606.014,32.155,176.033,-11441.645,0.000992,0.012192,-0.398438,-1.389679
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,...,1,89.730,383.203,41.999,1064.320,-37128.727,0.000500,0.000326,2.801361,-1.864105
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,...,1,89.943,113.770,4.724,612.181,-11849.360,0.276383,0.107635,-0.370117,1.146729
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,...,1,89.898,113.279,2.642,656.100,-11528.459,0.305247,0.123588,-0.404816,1.145599
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,...,1,89.871,603.311,21.027,519.267,-26964.873,0.000602,0.000387,1.423065,-1.873322
